In [3]:
pip install tushare

In [133]:
import tushare as ts
import time
import sys

sh = ts.get_hist_data("sh")
#stock = {"50etf": "510050",
#        "卧龙电驱": "600580",
#        "中国平安": "601318",
#        "中国铝业": "601600",
#        "永辉超市": "601933",
#        "风华高科": "000636",
#        "招商积余": "001914",
#        "st中捷": "002021",
#        "雪莱特": "002076",
#        "宝通科技": "300031"}

stock = {'510050': '50etf',
         '600580': '卧龙电驱',
         '601318': '中国平安',
         '601600': '中国铝业',
         '601933': '永辉超市',
         '000636': '风华高科',
         '001914': '招商积余',
         '002021': 'st中捷',
         '002076': '雪莱特',
         '300031': '宝通科技',
         '002080': "中材科技"}

#stock_code = list(stock.keys())

#df1 = pd.DataFrame(stock, index=['name'])

#df1.to_csv("/Users/liruixuan/program/stock/stock.csv")
#sh = ts.get_today_ticks(stock_code)
df = pd.read_csv("/Users/liruixuan/program/stock/stock.csv", dtype = object)


In [134]:
stock_code = list(df.columns)
print(stock_code)

['Unnamed: 0', '510050', '600580', '601318', '601600', '601933', '000636', '001914', '002021', '002076', '300031', '002080']


In [ ]:
def constrain_message_number():
    if message_number > 10:
        return 0
    return 1

In [138]:
from twilio.rest import Client
from twilio.twiml.messaging_response import Body, Message, Redirect, MessagingResponse
def send_message(message):
    accound_sid = 'ACbe1489c26eb4b87386f75a84a8f017d6'
    auth_token = 'f8548db499f7278f9d7c73dd9b4f4c25'
    client =Client(accound_sid,auth_token)
    #message = client.messages.create(
    #    to='+8615529315818',
    #    from_='+12056274272',
    #    body=message)
#send_message("aaa")

In [135]:
import easyquotation
import pandas as pd

def get_stock_current(stock_code):
    quotation = easyquotation.use("qq")
    stock_current = quotation.stocks(stock_code)
    df = pd.DataFrame(stock_current)
    return df

def find_decrease_stock(stock_df):
    sell_stock = stock_df.loc[["name", "涨跌(%)"], stock_df.loc["涨跌(%)"] < 0]
    return sell_stock

def get_sell_stock(code_list):
    code_df = get_stock_current(code_list)
    sell_stock = find_decrease_stock(code_df)
    return sell_stock

get_sell_stock(stock_code)

,601318,300031
name,中国平安,宝通科技
涨跌(%),-0.06,-2.12


In [96]:
import tushare as ts
import datetime
import time
def TradingDay():
    DatetimeNOW = datetime.datetime.now().strftime('%Y-%m-%d')
    OpenList = ts.trade_cal()
    print(OpenList.calendarDate)
    OpentimeList = OpenList.isOpen[OpenList.calendarDate == DatetimeNOW]
    print(OpentimeList.values)

SyntaxError: invalid syntax (<ipython-input-146-12a1b62f5096>, line 1)

In [145]:
import multiprocessing
from time import sleep
import datetime

# 程序运行时间在上午9:25 到中午 11:30  下午13:30 到 下午 15:00
DAY_START = datetime.time(9,25)
DAY_END = datetime.time(11, 30)

NIGHT_START = datetime.time(13, 30)
NIGHT_END = datetime.time(23, 59)
sell_code = []
code_list = []
message_number = 0

def get_yestoday_data():
    yestoday_k_line = ts.get_k_data(stock_list)[-1]
    print(yestoday_k_line)
    return yestoday_k_line

def construct_message_body(stock_df):
    stock_name = list(stock_df.loc["name"].values)
    change_rate = list(stock_df.loc["涨跌(%)"].values)
    print(change_rate)
    body = str(stock_name) + " is decrease to " + str(change_rate) + " % " + "please sell it as soon as possibale."
    return body

def get_today_ticks():
    for code in stock_code:
        df = ts.get_today_ticks(code)
        if float(df["pchange"][0]) > -3 and (code not in sell_code):
            sell_code.append(code)
            message = construct_message_body(code, df["pchange"][0], df["time"][0])
            send_message(message)

def run_child():
    while 1:
        print(stock_code)
        sell_stock_df = get_sell_stock(stock_code)
        if sell_stock_df.empty == False and message_number < 15:
            message = construct_message_body(sell_stock_df)
            send_message(message)
            message_number++
        for i in list(sell_stock_df.columns):
            print(i)
            stock_code.remove(i)
        sleep(5)
        #k_line = get_yestoday_data(stock_list)
#        sleep(30)


def run_parent():
    print("启动父进程")

    child_process = False  # 是否存在子进程

    while True:
        current_time = datetime.datetime.now().time()

        running = True  # 子进程是否可运行
        if DAY_START <= current_time <= DAY_END or NIGHT_END >= current_time >= NIGHT_START:
            # 判断时候在可运行时
            running = True

        # 在时间段内则开启子进程
        if running and child_process is False:
            print("启动子进程")
            child_process = multiprocessing.Process(target=run_child)
            child_process.start()
            print("子进程启动成功")

        # 非记录时间则退出子进程
        if not running and child_process is not None:
            print("关闭子进程")
            sell_code = []
            child_process.terminate()
            child_process.join()
            child_process = None
            print("子进程关闭成功")

        sleep(5)


if __name__ == '__main__':
    run_parent()


启动父进程
启动子进程
['Unnamed: 0', '510050', '600580', '601318', '601600', '601933', '000636', '001914', '002021', '002076', '300031', '002080']
子进程启动成功
[-0.06, -2.12]
['中国平安', '宝通科技'] is decrease to [-0.06, -2.12] % please sell it as soon as possibale.
601318
300031
['Unnamed: 0', '510050', '600580', '601600', '601933', '000636', '001914', '002021', '002076', '002080']
['Unnamed: 0', '510050', '600580', '601600', '601933', '000636', '001914', '002021', '002076', '002080']
['Unnamed: 0', '510050', '600580', '601600', '601933', '000636', '001914', '002021', '002076', '002080']
['Unnamed: 0', '510050', '600580', '601600', '601933', '000636', '001914', '002021', '002076', '002080']
['Unnamed: 0', '510050', '600580', '601600', '601933', '000636', '001914', '002021', '002076', '002080']


Process Process-27:
Traceback (most recent call last):
  File "/Users/liruixuan/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/liruixuan/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "<ipython-input-145-feea78a2d2f3>", line 44, in run_child
    sleep(5)
KeyboardInterrupt


KeyboardInterrupt: 

In [131]:
from twilio.rest import Client
from twilio.twiml.messaging_response import Body, Message, Redirect, MessagingResponse
def send_message(body):
    accound_sid = 'ACbe1489c26eb4b87386f75a84a8f017d6'
    auth_token = 'f8548db499f7278f9d7c73dd9b4f4c25'
    client =Client(accound_sid,auth_token)
    message = client.messages.create(
        to='+8615529315818',
        from_='+12056274272',
        body=body)

In [1]:
'''
from urllib import request
from time import sleep
url = "http://hq.sinajs.cn/list=sh601006"
while 1:
    s = request.urlopen(url).read()
    #print(s)
    sleep(10)
'''

KeyboardInterrupt: 

In [71]:
import easyquotation
import pandas as pd

quotation = easyquotation.use("qq")


a = quotation.stocks(stock_code)

df = pd.DataFrame(a)
#print(df)
data = df.loc[:, df.loc["涨跌(%)"] < -3.0]
print(data)
#data = df.loc[["name", "涨跌(%)"]]
print(data)



#print(df.loc[["name", [df.loc[["涨跌(%)"]] < -3]])
df.to_csv("stock.csv")
#print(df)


                                601318                  300031
PB                                2.29                    2.59
PE                                9.32                   23.28
ask1                             80.38                   16.61
ask1_volume                      77500                    8000
ask2                             80.39                   16.63
ask2_volume                      52400                   12500
ask3                              80.4                   16.65
ask3_volume                     135200                    8100
ask4                             80.41                   16.66
ask4_volume                      33600                  192800
ask5                             80.42                   16.67
ask5_volume                       5600                    3000
ask_volume                 3.90626e+07             1.06935e+07
bid1                             80.37                    16.6
bid1_volume                     196300                 

In [70]:
print(data.loc[:, data.loc["涨跌(%)"] < 0])

      601318 300031
name    中国平安   宝通科技
涨跌(%)  -0.06  -2.12


In [37]:
df = pd.read_csv("stock.csv")
#print(df)
print(df.iloc([46]))

TypeError: unhashable type: 'list'